# <center><b>Data Claiming</b></center>

<h2>Preparation</h2>

In [199]:
# include libraries
from requests_html import HTMLSession
import urllib.robotparser
import re
import requests
import requests_cache
import time, json
import pandas as pd
import random
import time
import string
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# target link
anchor_user = "one direction"
target_web_link = 'https://soundcloud.com'
test_cache_web_link1 = 'https://raw.githubusercontent.com/KienTrann/requests-cache-testing/main/should_be_cached.txt'
test_cache_web_link2 = 'https://raw.githubusercontent.com/KienTrann/requests-cache-testing/main/should_not_be_cached.txt'
chrome_driver_binary = "D:\Sequence\Comp-Suppoter\Chrome\chromedriver"

# Dont care about these lines, I just review the lesson of Lab01
(move to Hand on Crawling)

In [200]:
# get the website source
session = HTMLSession()
source = session.get('https://thinhsuy.github.io/VisualPage/#')
tag = source.html.find('.menu-a', first=True)
print(f"Tag's HTML string: {tag.html}")
print(f"Tag's text string: {tag.text}")
print(f"Attributes dictionary tag: {tag.attrs}")

Tag's HTML string: <a class="menu-a" href="#">About</a>
Tag's text string: About
Attributes dictionary tag: {'class': ('menu-a',), 'href': '#'}


In [201]:
# check the robot.txt of website
rp = urllib.robotparser.RobotFileParser()
rp.set_url(target_web_link+'/robots.txt')
rp.read()
rp.can_fetch('*', target_web_link)

True

In [202]:
# Regular Expression
re.findall(r'\w{5}', tag.text)

['About']

In [203]:
# Working with dynamic webpage
session = HTMLSession()
source = session.get(target_web_link)
try: source.html.render()
except: print('Not running .render() in Jupyter! Run it in powershell file.py instead')

Not running .render() in Jupyter! Run it in powershell file.py instead


<h2>Hand on crawling data</h2>

In [204]:
# Limit the total step that the request would crawl down data
def check_filter(response: requests.models.Response)->bool:
    return True if '"incomplete_result":true' in response.text else False

requests_cache.install_cache("my_cache", filter_fn=check_filter)

# Request function
sleep_time = 1
def get_request(url):
    r = requests.get(url)
    while not(r.ok):
        time.sleep(sleep_time)
        r = requests.get(url)
    return r

# Testing
print(f"Test from link 1: \n{get_request(test_cache_web_link1).text}")
print(f"Test from link 2: \n{get_request(test_cache_web_link2).text}")

Test from link 1: 
abc
xyz
"incomplete_results":false

Test from link 2: 
abc
xyz
"incomplete_results":true



In [205]:
def OpenWebBrowser(driver, link, by=By.ID, id_coockie_trust="none", delay_element_loading=10):
    time.sleep(sleep_time)
    driver.get(link)
    if id_coockie_trust=="none": return True
    try:
        WebDriverWait(driver, delay_element_loading).until(
            EC.presence_of_element_located(
                (by, id_coockie_trust)
            ))
    except TimeoutException: 
        print("Runtime out")
        return False
    time.sleep(sleep_time)
    driver.find_element(by, id_coockie_trust).click()
    return True

def ScrollScreen(driver, epochs=1, time_sleep=0.5):
    for _ in range(epochs):
        time.sleep(time_sleep)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

def getElements(driver, by, id, sleep_time=1):
    time.sleep(sleep_time)
    element = driver.find_elements(by, id)
    time.sleep(sleep_time*2)
    return element

def getRequest(driver, url, sleep_time=1):
    driver.get(url)
    time.sleep(sleep_time)

def WaitUntilPresense(driver, by, id, delay=5):
    try:
        WebDriverWait(driver, delay).until(EC.presence_of_element_located((by, id)))
        return True
    except TimeoutException: 
        print("Runtime out")
        return False

def addValue(dictionary, key, value, default='NaN'):
    try: dictionary[key] = value
    except: dictionary[key] = default
    return dictionary

# Get user information

In [206]:
def Get_user_infor(driver, user_dict):
    stats_value = getElements(driver, By.CLASS_NAME, "infoStats__value")
    user_dict = addValue(user_dict, 'Followers', stats_value[0].text)
    user_dict = addValue(user_dict, 'Following', stats_value[1].text)
    user_dict = addValue(user_dict, 'Tracks', stats_value[2].text)
    lasts_value = getElements(driver, By.CLASS_NAME,'sc-visuallyhidden')
    user_dict = addValue(user_dict, 'Last_action', lasts_value[0].text)
    user_dict = addValue(user_dict, 'Last_track_played', lasts_value[1].text)
    return user_dict

# Get Playlist information

In [207]:
def Get_playlist_infor(driver, playlist_dict):
    stats = getElements(driver, By.CLASS_NAME, 'sc-link-secondary')
    playlist_dict = addValue(playlist_dict, 'Singer', stats[0].text)
    playlist_dict = addValue(playlist_dict, 'Likes', str(stats[1].text).split('\n')[1])
    playlist_dict = addValue(playlist_dict, 'Reposts', str(stats[2].text).split('\n')[1])
    last = getElements(driver, By.CLASS_NAME, 'sc-visuallyhidden')
    playlist_dict = addValue(playlist_dict, 'Release', last[0].text)
    ScrollScreen(driver, epochs=5)
    total_track = getElements(driver, By.CSS_SELECTOR, '.trackList__item.sc-border-light-bottom.sc-px-2x')
    playlist_dict = addValue(playlist_dict, 'Total_tracks', len(total_track))
    return playlist_dict

# Get Track information

In [208]:
def Get_track_infor(driver, track_dict):
    ScrollScreen(driver)
    stats = getElements(driver, By.TAG_NAME, 'span')
    try:
        for i in range(len(stats)):
            if stats[i].text == 'View all likes':
                track_dict = addValue(track_dict, 'Release', stats[i-7].text)
                track_dict = addValue(track_dict, 'Plays', stats[i-2].text)
                track_dict = addValue(track_dict, 'Likes', stats[i+1].text)
                track_dict = addValue(track_dict, 'Reposts', stats[i+3].text)
                break
    except: print('Exception tracks occured!')
    return track_dict

# General Crawling Step by Step

In [209]:
def Get_one_row_information(driver, sleep_time=1, iter=0, list_order=list(string.ascii_lowercase)):
    playlist_dict, track_dict, user_dict = dict(), dict(), dict()
    ScrollScreen(driver, epochs=2)
    # get infor by search and choose the first one as PLAYLIST
    playlist_list_url = getElements(driver, By.CSS_SELECTOR, '.sc-link-primary.soundTitle__title.sc-link-dark.sc-text-h4')
    playlist_dict = addValue(playlist_dict, 'Name', playlist_list_url[iter].text)
    getRequest(driver, playlist_list_url[iter].get_attribute('href'))
    playlist_dict = Get_playlist_infor(driver, playlist_dict)
    
    # get random track from playlist for get specific infor TRACK
    track_list_url = getElements(driver, By.CLASS_NAME, 'trackItem__trackTitle')
    iter = random.randint(0, len(track_list_url)-1)
    track_dict = addValue(track_dict, 'Name', track_list_url[iter].text)
    getRequest(driver, track_list_url[iter].get_attribute('href'))
    track_dict = Get_track_infor(driver, track_dict)

    # get directly to the profile of current user
    user_list_url = getElements(driver, By.CLASS_NAME,'sc-link-secondary')
    user_dict = addValue(user_dict, 'Name', user_list_url[0].text)
    getRequest(driver, user_list_url[0].get_attribute('href'))
    user_dict = Get_user_infor(driver, user_dict)
    return [playlist_dict, track_dict, user_dict]

In [214]:
driver = webdriver.Chrome()
df_playlist = pd.DataFrame(columns=['Name', 'Singer', 'Likes', 'Reposts', 'Release', 'Total_tracks'])
df_track = pd.DataFrame(columns=['Name', 'Realease', 'Plays', 'Likes', 'Reposts'])
df_user = pd.DataFrame(columns=['Name', 'Followers', 'Following', 'Tracks', 'Last_action', 'Last_track_played'])
alphabet = list(string.ascii_lowercase)
data = []
# run throw and search all the alphabet then choose 1 in them to check (24 alphabets -> 24 rows currently), loop 1 more loop for 40 cases of each alphabet (later)
for char in alphabet:
    url = target_web_link + '/search/sets?q=' + char
    if (OpenWebBrowser(driver, url, id_coockie_trust="onetrust-accept-btn-handler")):
        data.append(Get_one_row_information(driver))
    # test case on 1 alphabet first
    break
driver.close()

In [215]:
data

[[{'Name': 'Acid Jazz',
   'Singer': 'TMN Playlisted',
   'Likes': '388K',
   'Reposts': '35.8K',
   'Release': '6 years ago',
   'Total_tracks': 50},
  {'Name': '119',
   'Release': '6 years ago',
   'Plays': '5,045,518 plays',
   'Likes': '34.6K',
   'Reposts': '1,220'},
  {'Name': 'TJCK',
   'Followers': '898',
   'Following': '30',
   'Tracks': '4',
   'Last_action': 'Reposted 3 months ago',
   'Last_track_played': '94 plays'}]]